In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# IMPORT STATEMENTS...
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler

#LOADING DATA...
with np.load("/kaggle/input/watai-open-project-f23-beginner-classical-ml/beginner_data_cml.npz") as data:
    X_train = data['train_images']  # Shape (8981, 64, 64, 3)
    y_train = data['train_labels']  # Shape (8981,)
    X_test = data['test_images']  # Shape (1800, 64, 64, 3)
    # we don't have y_test because the labels for the test is the basis of the scoring in the competition...
    
#creating target vectors for the classification task...
y_train_1 = (y_train == 1)

In [ ]:
# Reshape the image data to 2D
X_train_flattened = X_train.reshape(X_train.shape[0], -1)  # Flatten each image
# Make sure to apply the same reshaping procedure to the test data when you want to make predictions on it.

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_flattened)

print("Fitting SVM classifier...")
clf = SVC(kernel='linear', random_state=42)
clf.fit(X_train_flattened, y_train_1)
print("SVM classifier trained.")

In [ ]:
# Use cross_val_predict with SVM to get decision_function scores
print("Getting decision_function scores...")
y_scores_svm = cross_val_predict(clf, X_train_flattened, y_train_1, cv=3, method="decision_function")
print("Decision_function scores obtained.")

In [ ]:
#PLOTTING DA MODEL
# y_scores_svm_ = y_probas_forest[:, 1] // what does this line do?
precisions_svm, recalls_svm, thresholds_svm = precision_recall_curve(y_train_1, y_scores_svm)
plt.plot(recalls_svm, precisions_svm, "b-", linewidth=2, label="SVM")
plt.show()

In [ ]:
# Calculate F1 score, ROC AUC, precision, and recall
y_pred_svm = y_scores_svm >= 0  # You can adjust the threshold here
f1_svm = f1_score(y_train_1, y_pred_svm)
roc_auc_svm = roc_auc_score(y_train_1, y_scores_svm)
precision_svm = precision_score(y_train_1, y_pred_svm)
recall_svm = recall_score(y_train_1, y_pred_svm)

print("F1 Score (SVM):", f1_svm)
print("ROC AUC (SVM):", roc_auc_svm)
print("Precision (SVM):", precision_svm)
print("Recall (SVM):", recall_svm)

In [ ]:
# Reshape the image data for testing
X_test_flattened = X_test.reshape(X_test.shape[0], -1)

# Make predictions with SVM
predictions_svm = clf.predict(X_test_flattened)

N_test = X_test.shape[0]
df_svm = pd.DataFrame({
    "Index": np.arange(N_test),
    "Label": predictions_svm
})
df_svm.to_csv("submission.csv", index=False)